In [1]:
# Необходимые импорты
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import time
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
from typing import List

In [2]:
# сбор всей необходимой информации с карточки товара
def get_info_for_card(url: str):
        br = webdriver.Chrome()
        br.get(url)
        time.sleep(5)
        br.execute_script("window.scrollTo(0, 1620)")
        time.sleep(5)
        page_add = br.page_source
        tree = BeautifulSoup(page_add, 'html')
        br.close()
        
        image_card = tree.find_all('div', {'class': 'slide__content img-plug'})
        image_card = image_card[0].img['src']

        image_feedbacks = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
        image_feedbacks = image_feedbacks[0].img['src']
        
        return image_card, image_feedbacks


# скачивание картинки
def load_image(url, path, name):
    img = requests.get(url)
    full_path = path / name
    
    with open(full_path, "wb") as out:
        out.write(img.content)

In [19]:
def parsing_items_wb(
    urls: List[str],
    data_dir: Path,
    start_category: int = 0,
    start_item: int = 0):
#     try:
        for i in range(start_category, len(urls)):
            main_url = urls[i]
            br = webdriver.Chrome()
            br.get(main_url)
            time.sleep(5)
            page_add = br.page_source
            tree = BeautifulSoup(page_add, 'html')
            br.close()
            
            # берем только 15 первых товаров
            items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
            items_href = [items[i]['href'] for i in range(len(items))][:15]
            
            for j in tqdm(range(start_item, len(items_href))):
#                 print(items_href[j])
                image_card, image_feedbacks = get_info_for_card(items_href[j])
                
                path_card = data_dir / Path(main_url).name / "card"
                path_feedbacks = data_dir / Path(main_url).name / "feedbacks"
                
                if j==0:
                    path_card.mkdir(parents=True, exist_ok=True)
                    path_feedbacks.mkdir(parents=True, exist_ok=True)
                
                load_image(url=image_card, path=path_card, name=f"{j}.png")
                load_image(url=image_feedbacks, path=path_feedbacks, name=f"{j}_feedbacks.png")
            
            print(f'Прошла категория "{Path(main_url).name}"')
                
#     except Exception as e:
#         print(e)        

In [4]:
categories = pd.read_csv("wb_categories.csv")
categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...


In [5]:
data_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")

In [23]:
parsing_items_wb(urls=["https://www.wildberries.ru/catalog/igrushki/antistress"],
                 data_dir=data_dir,
                 start_category=0,
                 start_item=14)

  0%|          | 0/1 [00:00<?, ?it/s]

Прошла категория "antistress"


# Тестирование

In [2]:
url = 'https://www.wildberries.ru/catalog/74299675/detail.aspx'
url = 'https://www.wildberries.ru/catalog/149410773/detail.aspx'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
br.execute_script("window.scrollTo(0, 1620)")
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [7]:
# submit = br.find_element(By.CLASS_NAME, 'product-page__btn-detail j-wba-card-item j-wba-card-item-show j-wba-card-item-observe')
# submit

In [74]:
tree

<html class="adaptive" lang="ru" translate="no"><head><style class="vjs-styles-defaults">
      .video-js {
        width: 300px;
        height: 150px;
      }

      .vjs-fluid:not(.vjs-audio-only-mode) {
        padding-top: 56.25%
      }
    </style>
<meta charset="utf-8"/>
<base href="https://www.wildberries.ru/"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="no-referrer-when-downgrade" name="referrer"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Wildberries" name="application-name"/>
<meta content="61b95d33285e2612" name="yandex-verification"/>
<meta content="guMau7oNvBifqUhZGuuGVtiGD0HkKe8v6uwYpvj2ZIg" name="google-site-verification"/>
<meta content="e08a4d66746555c8641fe953d12262c8" name="wmail-verification"/>
<meta content="app-id=597880187" name="apple-itunes-app"/>
<link href="/manifestv2.json" rel="manifest"/>
<link href="/opensearch.xml" rel="search" title="test" type="application/opensearchdescription+xml

In [68]:
data = tree.find_all('div', {'class': 'slide__content img-plug'})
data[0].img['src']

'https://basket-10.wbbasket.ru/vol1494/part149410/149410773/images/big/1.webp'

In [69]:
data = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
data[0].img['src']

'https://feedback05.wbbasket.ru/vol1612/part161219/161219161/photos/ms.webp'

___

In [26]:
url = 'https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [33]:
data = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
print(len(data))
data[0]['href']

30


'https://www.wildberries.ru/catalog/160832044/detail.aspx'

In [39]:
items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
items_href = [items[i]['href'] for i in range(len(items))][:15]

___

In [17]:
categories = pd.read_csv("wb_categories.csv")
categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...


In [24]:
Path(categories.iloc[0].url).name

'bluzki-i-rubashki'

In [25]:
categories["url"]

0    https://www.wildberries.ru/catalog/zhenshchina...
1    https://www.wildberries.ru/catalog/zhenshchina...
Name: url, dtype: object

___

In [10]:
# # скачивание картинки
# img = "https://basket-05.wbbasket.ru/vol742/part74299/74299675/images/c246x328/1.webp"

# p = requests.get(img)

# with open("data/img1.png", "wb") as out:
#     out.write(p.content)

In [ ]:
data_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")